In [135]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import csv
from requests import post
import numpy as np

In [136]:
steam_data =pd.read_csv(r"C:\Users\valdi\OneDrive\Documents\Springboard\steam.csv")

In [137]:
print(steam_data.info)

<bound method DataFrame.info of          appid                        name release_date  english  \
0           10              Counter-Strike   2000-11-01        1   
1           20       Team Fortress Classic   1999-04-01        1   
2           30               Day of Defeat   2003-05-01        1   
3           40          Deathmatch Classic   2001-06-01        1   
4           50   Half-Life: Opposing Force   1999-11-01        1   
...        ...                         ...          ...      ...   
27070  1065230             Room of Pandora   2019-04-24        1   
27071  1065570                   Cyber Gun   2019-04-23        1   
27072  1065650            Super Star Blast   2019-04-24        1   
27073  1066700  New Yankee 7: Deer Hunters   2019-04-17        1   
27074  1069460                   Rune Lord   2019-04-24        1   

                developer             publisher          platforms  \
0                   Valve                 Valve  windows;mac;linux   
1          

In [151]:
organized_df= steam_data[['name', 'categories', 'genres', 'positive_ratings', 'negative_ratings']].copy()
print(organized_df.head())

                        name  \
0             Counter-Strike   
1      Team Fortress Classic   
2              Day of Defeat   
3         Deathmatch Classic   
4  Half-Life: Opposing Force   

                                          categories  genres  \
0  Multi-player;Online Multi-Player;Local Multi-P...  Action   
1  Multi-player;Online Multi-Player;Local Multi-P...  Action   
2              Multi-player;Valve Anti-Cheat enabled  Action   
3  Multi-player;Online Multi-Player;Local Multi-P...  Action   
4  Single-player;Multi-player;Valve Anti-Cheat en...  Action   

   positive_ratings  negative_ratings  
0            124534              3339  
1              3318               633  
2              3416               398  
3              1273               267  
4              5250               288  


In [176]:
organized_df.duplicated(subset= ['name']).sum()
organized_df.loc[organized_df.duplicated(['name']), :]
minus_repeating_games=organized_df.drop_duplicates(subset= 'name',keep='first')
print(minus_repeating_games)

                             name  \
0                  Counter-Strike   
1           Team Fortress Classic   
2                   Day of Defeat   
3              Deathmatch Classic   
4       Half-Life: Opposing Force   
...                           ...   
27070             Room of Pandora   
27071                   Cyber Gun   
27072            Super Star Blast   
27073  New Yankee 7: Deer Hunters   
27074                   Rune Lord   

                                              categories  \
0      Multi-player;Online Multi-Player;Local Multi-P...   
1      Multi-player;Online Multi-Player;Local Multi-P...   
2                  Multi-player;Valve Anti-Cheat enabled   
3      Multi-player;Online Multi-Player;Local Multi-P...   
4      Single-player;Multi-player;Valve Anti-Cheat en...   
...                                                  ...   
27070                   Single-player;Steam Achievements   
27071                                      Single-player   
27072  Single-p

In [177]:
minus_repeating_games['positive_ratings'].replace(0,'None')

0        124534
1          3318
2          3416
3          1273
4          5250
          ...  
27070         3
27071         8
27072      None
27073         2
27074         4
Name: positive_ratings, Length: 27033, dtype: object

In [178]:
minus_repeating_games['negative_ratings'].replace(0,'None')

0        3339
1         633
2         398
3         267
4         288
         ... 
27070    None
27071       1
27072       1
27073    None
27074    None
Name: negative_ratings, Length: 27033, dtype: object

In [179]:
missing_organized_df = pd.isnull(minus_repeating_games['name']).sum()
print(missing_organized_df)

0


In [180]:
print(minus_repeating_games['categories'].str.contains('Single-player').value_counts()[True])
print(minus_repeating_games['categories'].str.contains('Multi-player').value_counts()[True])
# The Multiplayer .value_counts() would include categories such as Local Multiplayer and Cross-platform Multiplayer 

25637
3971


In [181]:
print(minus_repeating_games['genres'].str.contains('Indie').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Action').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Casual').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Adventure').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Strategy').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Simulation').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('RPG').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Free to Play').value_counts()[True])
print(minus_repeating_games['genres'].str.contains('Racing').value_counts()[True])
#there should be more data in genres than rows due to games being able to be classified as more than just one genre 

19392
11884
10190
10017
5243
5180
4304
1702
1022


In [184]:
highest_rating_game= minus_repeating_games.groupby('name')['positive_ratings'].max().reset_index().sort_values(['positive_ratings'], ascending =False)
print(highest_rating_game)

                                   name  positive_ratings
4802   Counter-Strike: Global Offensive           2644404
6581                             Dota 2            863507
21901                   Team Fortress 2            515879
15971     PLAYERUNKNOWN'S BATTLEGROUNDS            496184
9168                        Garry's Mod            363721
...                                 ...               ...
25000                  Virtual Ninja VR                 0
8639      Forgotten Sound 1: Revelation                 0
25882       World of Virtual Reality VR                 0
2221                   Barrimean Jungle                 0
23839        Towards a perilous journey                 0

[27033 rows x 2 columns]


In [191]:
lowest_rating_game= minus_repeating_games.groupby('name')['negative_ratings'].max().reset_index().sort_values(['negative_ratings'], ascending =False)
print(lowest_rating_game)

                                   name  negative_ratings
15971     PLAYERUNKNOWN'S BATTLEGROUNDS            487076
4802   Counter-Strike: Global Offensive            402313
6581                             Dota 2            142079
9611                 Grand Theft Auto V            139308
26183                  Z1 Battle Royale             91664
...                                 ...               ...
22108                  The Angry Banana                 0
6627                       Dr. Spacezoo                 0
17371                         R Academy                 0
17370                              Qybe                 0
15341                          Nurikabe                 0

[27033 rows x 2 columns]
